# Build Fct Tables

I started by setting up the boxscore and scoring data tables in 'game_stats.ipynb'

This file will be for the other fact tables. Currently one dedicated to betting/gambling odds. Another for the record of each team.

## Imports

In [1]:
import pandas as pd
import json
import os
import requests

from dotenv import load_dotenv

## API Credentials

In [2]:
#Get API Key and host
load_dotenv()
api_token = os.getenv('nfl_api_key')
api_host = os.getenv('rapid_api_host')

#rapidapi headers
headers = {
	"X-RapidAPI-Key": "{key}".format(key=api_token),
	"X-RapidAPI-Host": "{host}".format(host=api_host)
}

## Betting

For this the endpoint is 'Get NFL Betting Odds'. I will use to to save off the different spreads and odds for each game, by vendor.

It takes the game id to pull, so we will have to loop through each game to pull all the records. Here I'll test with one game.

Unfortunately, it looks like only 2023 data for this is available at this point.

### Extract

In [42]:
#re-create sportbook mapping in dim_tables. This will be import in the final product.
sportsbook_dict = {
    "betmgm": 1,
    "bet365": 2,
    "fanduel": 3,
    "wynnbet": 4,
    "unibet": 5,
    "pointsbet": 6,
    "betrivers": 7,
    "ceasars_sportsbook": 8,
    "draftkings": 9   
}

In [13]:
#API Params
#API Endpoint for NFL Team Info
url = "https://tank01-nfl-live-in-game-real-time-statistics-nfl.p.rapidapi.com/getNFLBettingOdds"

#pull in only main team info, none of the rosters, schedules, stats, etc.
querystring = querystring = {"gameID":"20231001_ARI@SF"}

In [14]:
#get team data
response = requests.get(url, headers=headers, params=querystring)

In [24]:
response.json()['body']['20231001_ARI@SF']['betmgm']

{'totalUnder': '44',
 'awayTeamSpread': '+14',
 'awayTeamSpreadOdds': '-110',
 'homeTeamSpread': '-14',
 'homeTeamSpreadOdds': '-110',
 'totalOverOdds': '-110',
 'totalUnderOdds': '-110',
 'awayTeamMLOdds': '+625',
 'homeTeamMLOdds': '-1000',
 'totalOver': '44'}

### Transform

This will be fun - I think I need to iterrate through the key value pairs in the sportsbook dict to save the primary key and drill into each record.

I will also need to include the game id from the API call itself and the timestamp from the root of the body.

In the future if more get added to the API I'll need to catch it and add to the list. Maybe by checking the number of records compared to our dim_sportsbook table.

In [37]:
#setup body of json
game_odds = response.json()['body']['20231001_ARI@SF']

In [38]:
#view path for updated time
game_odds['last_updated_e_time']

'1695954490.0765254'

In [39]:
#view path for a sportsbook
game_odds['betmgm']

{'totalUnder': '44',
 'awayTeamSpread': '+14',
 'awayTeamSpreadOdds': '-110',
 'homeTeamSpread': '-14',
 'homeTeamSpreadOdds': '-110',
 'totalOverOdds': '-110',
 'totalUnderOdds': '-110',
 'awayTeamMLOdds': '+625',
 'homeTeamMLOdds': '-1000',
 'totalOver': '44'}

In [67]:
#setup empty list for savings the data in individual records
betting_data_list = []

In [68]:
#loop through sportsbook dict to create data for each game
for key, value in sportsbook_dict.items():
    
    #set updated time
    updated_time = game_odds['last_updated_e_time']

    try: #try to grab data for each sportsbook, but pass if that sportsbook doesn't have a record
        #drill into sportsbook
        sportsbook_data = game_odds['{sportsbook}'.format(sportsbook=key)]

        #create dictionary to append to list
        betting_odds = {
            "game_id": "20231001_ARI@SF",
            "last_update": updated_time,
            "sportsbook_id": value,
            "total_over_under": sportsbook_data['totalOver'],
            "over_odds": sportsbook_data['totalOverOdds'],
            "under_odds": sportsbook_data['totalUnderOdds'],
            "home_spread": sportsbook_data['homeTeamSpread'],
            "home_odds": sportsbook_data['homeTeamSpreadOdds'],
            "away_spread": sportsbook_data['awayTeamSpread'],
            "away_odds": sportsbook_data['awayTeamSpreadOdds'],
            "home_ml_odds": sportsbook_data['homeTeamMLOdds'],
            "away_ml_odds": sportsbook_data['awayTeamMLOdds']
        }

        betting_data_list.append(betting_odds)
    
    except KeyError:
        pass

In [71]:
betting_data_list[:2]

[{'game_id': '20231001_ARI@SF',
  'last_update': '1695954490.0765254',
  'sportsbook_id': 1,
  'total_over_under': '44',
  'over_odds': '-110',
  'under_odds': '-110',
  'home_spread': '-14',
  'home_odds': '-110',
  'away_spread': '+14',
  'away_odds': '-110',
  'home_ml_odds': '-1000',
  'away_ml_odds': '+625'},
 {'game_id': '20231001_ARI@SF',
  'last_update': '1695954490.0765254',
  'sportsbook_id': 2,
  'total_over_under': '44',
  'over_odds': '-110',
  'under_odds': '-110',
  'home_spread': '-14',
  'home_odds': '-110',
  'away_spread': '+14',
  'away_odds': '-110',
  'home_ml_odds': '-850',
  'away_ml_odds': '+575'}]

That looks pretty good for now.

## Record